<a href="https://colab.research.google.com/github/imabari/hyougo/blob/master/hyougo_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install jsonschema

# ダウンロード

In [0]:
!wget --tries=3 https://web.pref.hyogo.lg.jp/kk03/documents/pcr.xlsx -O pcr.xlsx

--2020-03-29 13:21:51--  https://web.pref.hyogo.lg.jp/kk03/documents/pcr.xlsx
Resolving web.pref.hyogo.lg.jp (web.pref.hyogo.lg.jp)... 210.130.236.38
Connecting to web.pref.hyogo.lg.jp (web.pref.hyogo.lg.jp)|210.130.236.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10058 (9.8K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘pcr.xlsx’

pcr.xlsx            100%[===================>]   9.82K  --.-KB/s    in 0s      

2020-03-29 13:21:58 (97.1 MB/s) - ‘pcr.xlsx’ saved [10058/10058]



In [0]:
!wget --tries=3 https://web.pref.hyogo.lg.jp/kk03/documents/yousei.xlsx -O yousei.xlsx

--2020-03-29 13:22:00--  https://web.pref.hyogo.lg.jp/kk03/documents/yousei.xlsx
Resolving web.pref.hyogo.lg.jp (web.pref.hyogo.lg.jp)... 210.130.236.38
Connecting to web.pref.hyogo.lg.jp (web.pref.hyogo.lg.jp)|210.130.236.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11750 (11K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘yousei.xlsx’

yousei.xlsx         100%[===================>]  11.47K  --.-KB/s    in 0s      

2020-03-29 13:22:01 (144 MB/s) - ‘yousei.xlsx’ saved [11750/11750]



In [0]:
#!wget --tries=3 https://web.pref.hyogo.lg.jp/kk03/documents/$(date +%m%d --date '1 day ago')coronakanjya.xlsx

--2020-03-29 13:22:03--  https://web.pref.hyogo.lg.jp/kk03/documents/0327coronakanjya.xlsx
Resolving web.pref.hyogo.lg.jp (web.pref.hyogo.lg.jp)... 210.130.236.38
Connecting to web.pref.hyogo.lg.jp (web.pref.hyogo.lg.jp)|210.130.236.38|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2020-03-29 13:22:09 ERROR 404: Not Found.



In [0]:
import requests
from bs4 import BeautifulSoup

from urllib.parse import urljoin

In [0]:
url = "https://web.pref.hyogo.lg.jp/kk03/corona_kanjyajyokyo.html"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko"
}

In [0]:
r = requests.get(url, headers=headers)

r.raise_for_status()

soup = BeautifulSoup(r.content, "html.parser")

In [0]:
tag = soup.select_one("a.icon_excel")

s = tag.get_text(strip=True)

link = urljoin(url, tag.get("href"))

link

'https://web.pref.hyogo.lg.jp/kk03/documents/0328corona_kanjya.xlsx'

In [0]:
!wget --tries=3 $link -O kanja.xlsx

--2020-03-29 13:22:21--  https://web.pref.hyogo.lg.jp/kk03/documents/0328corona_kanjya.xlsx
Resolving web.pref.hyogo.lg.jp (web.pref.hyogo.lg.jp)... 210.130.236.38
Connecting to web.pref.hyogo.lg.jp (web.pref.hyogo.lg.jp)|210.130.236.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30040 (29K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘kanja.xlsx’

kanja.xlsx          100%[===================>]  29.34K   179KB/s    in 0.2s    

2020-03-29 13:22:37 (179 KB/s) - ‘kanja.xlsx’ saved [30040/30040]



# SCHEMA

In [0]:
INSPECTIONS_SUMMARY_SCHEMA = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "data": {
            "type": "object",
            "additionalProperties": {
                "type": "array",
                "items": {"default": 0, "type": "integer"},
            },
        },
        "labels": {
            "type": "array",
            "items": {"pattern": r"^[0-9]{2}/[0-9]{2}$", "type": "string"},
        },
        "last_update": {
            "pattern": r"^[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}$",
            "type": "string",
        },
    },
    "required": ["data", "labels", "last_update"],
}

In [0]:
PATIENTS_SUMMARY_SCHEMA = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "data": {
            "type": "array",
            "items": {
                "type": "object",
                "additionalProperties": False,
                "properties": {
                    "日付": {"type": "string", "format": "date"},
                    "小計": {"default": 0, "type": "integer"},
                },
                "required": ["小計", "日付"],
            },
        },
        "last_update": {
            "pattern": r"^[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}$",
            "type": "string",
        },
    },
    "required": ["data", "last_update"],
}

In [0]:
AGE_SCHEMA = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "data": {
            "type": "object",
            "additionalProperties": {
                "default": 0,
                "type": "integer"
            }
        },
        "last_update": {
            "pattern": r"^[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}$",
            "type": "string"
        }
    },
    "required": [
        "data",
        "last_update"
    ],
}

In [0]:
AGE_SUMMARY_SCHEMA = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "data": {
            "type": "object",
            "additionalProperties": {
                "type": "array",
                "items": {"default": 0, "type": "integer"},
            },
        },
        "labels": {
            "type": "array",
            "items": {"pattern": "^[0-9]{2}/[0-9]{2}$", "type": "string"},
        },
        "last_update": {
            "pattern": r"^[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}$",
            "type": "string",
        },
    },
    "required": ["data", "labels", "last_update"],
}

In [0]:
PATIENTS_SUMMARY_SCHEMA = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "data": {
            "type": "array",
            "items": {
                "type": "object",
                "additionalProperties": False,
                "properties": {
                    "日付": {"type": "string", "format": "date"},
                    "小計": {"default": 0, "type": "integer"},
                },
                "required": ["小計", "日付"],
            },
        },
        "last_update": {
            "pattern": r"^[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}$",
            "type": "string",
        },
    },
    "required": ["data", "last_update"],
}

In [0]:
PATIENTS_SCHEMA = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "$ref": "#/definitions/Main",
    "definitions": {
        "Main": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "data": {"type": "array", "items": {"$ref": "#/definitions/Datum"}},
                "last_update": {"type": "string"},
            },
            "required": ["data", "last_update"],
            "title": "Main",
        },
        "Datum": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "番号": {"type": "integer"},
                "居住地": {"type": "string"},
                "年代": {"$ref": "#/definitions/Age"},
                "性別": {"$ref": "#/definitions/Sex"},
                "備考": {"type": "string"},
                "退院": {"type": "null"},
                "date": {"type": "string", "format": "date"},
                "リリース日": {"type": "string", "format": "date-time"},
                "曜日": {"$ref": "#/definitions/Week"},
            },
            "required": ["date", "リリース日", "備考", "居住地", "年代", "性別", "曜日", "番号", "退院"],
            "title": "Datum",
        },
        "Age": {
            "type": "string",
            "enum": [
                "0代",
                "10代",
                "20代",
                "30代",
                "40代",
                "50代",
                "60代",
                "70代",
                "80代",
                "90代",
            ],
            "title": "Age",
        },
        "Sex": {"type": "string", "enum": ["男性", "女性"], "title": "Sex"},
        "Week": {
            "type": "string",
            "enum": ["月", "火", "水", "木", "金", "土", "日"],
            "title": "Week",
        },
    },
}

In [0]:
CLUSTERS_SUMMARY_SCHEMA = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "data": {
            "type": "object",
            "additionalProperties": {"default": 0, "type": "integer"},
        },
        "last_update": {
            "pattern": r"^[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}$",
            "type": "string",
        },
    },
    "required": ["data", "last_update"],
}

In [0]:
MAIN_SUMMARY_SCHEMA = {
    "$schema": "http://json-schema.org/draft-06/schema#",
    "$ref": "#/definitions/Main",
    "definitions": {
        "Main": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "attr": {"type": "string"},
                "value": {"type": "integer", "default": 0},
                "children": {
                    "type": "array",
                    "items": {"$ref": "#/definitions/Inspections"},
                },
                "last_update": {
                    "pattern": "^[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}$",
                    "type": "string",
                },
            },
            "required": ["attr", "children", "last_update", "value"],
            "title": "Main",
        },
        "Inspections": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "attr": {"type": "string"},
                "value": {"type": "integer", "default": 0},
                "children": {
                    "type": "array",
                    "items": {"$ref": "#/definitions/Patients"},
                },
            },
            "required": ["attr", "children", "value"],
            "title": "Inspections",
        },
        "Patients": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "attr": {"type": "string"},
                "value": {"type": "integer", "default": 0},
                "children": {
                    "type": "array",
                    "items": {"$ref": "#/definitions/Symptoms"},
                },
            },
            "required": ["attr", "value"],
            "title": "Patients",
        },
        "Symptoms": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "attr": {"type": "string"},
                "value": {"type": "integer", "default": 0},
            },
            "required": ["attr", "value"],
            "title": "Symptoms",
        },
    },
}

# データラングリング

In [0]:
import datetime
import json
import re
from urllib.parse import urljoin

import jsonschema
import pandas as pd

In [0]:
def dumps_json(file_name, json_data):

    with open(file_name, "w") as fw:
        json.dump(json_data, fw, ensure_ascii=False, indent=4)

In [0]:
def json_check(fn, d):

    r = requests.get(
        "https://raw.githubusercontent.com/stop-covid19-hyogo/covid19/development/data/"
        + fn
    )

    result = d == r.json()

    if not result:

        print(fn)

In [0]:
# 最終更新日
last_update = datetime.date.today()
last_update

datetime.date(2020, 3, 29)

## pcr.xlsx

In [0]:
df_pcr = pd.read_excel("pcr.xlsx", index_col="年月日").fillna(0).astype(int)

df_pcr["日付"] = df_pcr.index.strftime("%Y-%m-%d")

df_pcr.rename(columns={"PCR検査件数": "検査検体数", "陽性件数": "陽性確認"}, inplace=True)

In [0]:
# inspections_summary

df_insp = df_pcr.loc[:, ["検査検体数", "陽性確認"]].copy()

labels = df_insp.index.strftime("%m/%d")

In [0]:
inspections_summary = {
    "data": df_insp.to_dict(orient="list"),
    "labels": labels.tolist(),
    "last_update": last_update.strftime("%Y/%m/%d %H:%M"),
}

In [0]:
jsonschema.validate(inspections_summary, INSPECTIONS_SUMMARY_SCHEMA)

dumps_json("inspections_summary.json", inspections_summary)

In [0]:
# parent_summary

df_pts = df_pcr.loc[:, ["日付", "陽性確認"]].copy()

df_pts.rename(columns={"陽性確認": "小計"}, inplace=True)

In [0]:
patients_summary = {
    "data": df_pts.to_dict(orient="records"),
    "last_update": last_update.strftime("%Y/%m/%d %H:%M"),
}

In [0]:
jsonschema.validate(patients_summary, PATIENTS_SUMMARY_SCHEMA)

dumps_json("patients_summary.json", patients_summary)

## kanja.xlsx

In [0]:
df_tmp = pd.read_excel("kanja.xlsx", skiprows=3)

df_kanja = df_tmp[df_tmp["番号"].notnull()].copy()

df_kanja.dropna(how="all", axis=1, inplace=True)

df_kanja.columns = df_kanja.columns.map(lambda s: s.replace("\n", ""))

df_kanja["番号"] = df_kanja["番号"].astype("int64")
df_kanja["年代"] = df_kanja["年代"].astype("int64")

df_kanja["確認日"] = df_kanja["確認日"].apply(
    lambda date: pd.to_datetime(date, unit="D", origin=pd.Timestamp("1899/12/30"))
)

df_kanja["備考欄"] = df_kanja["備考欄"].str.replace("\n", "")

df_kanja.set_index("番号", inplace=True)

In [0]:
df_kanja.to_csv("kanja.tsv", sep="\t")

In [0]:
# 陽性患者数（累計）

len(df_kanja)

126

In [0]:
# 陽性患者数（日別）

df_pts = (
    df_kanja["確認日"]
    .value_counts()
    .sort_index()
    .asfreq("D", fill_value=0)
    .reset_index()
)

df_pts["日付"] = df_pts["index"].dt.strftime("%Y-%m-%d")

df_pts.rename(columns={"確認日": "小計"}, inplace=True)

df_pts.drop("index", axis=1, inplace=True)

df_pts

,小計,日付
0,1,2020-03-01
1,0,2020-03-02
2,2,2020-03-03
3,0,2020-03-04
4,1,2020-03-05
5,4,2020-03-06
6,2,2020-03-07
7,2,2020-03-08
8,4,2020-03-09
9,9,2020-03-10


In [0]:
patients_summary = {
    "data": df_pts.to_dict(orient="records"),
    "last_update": last_update.strftime("%Y/%m/%d %H:%M"),
}

In [0]:
jsonschema.validate(patients_summary, PATIENTS_SUMMARY_SCHEMA)

dumps_json("patients_summary.json", patients_summary)

In [0]:
# 陽性患者情報

df_pt = (
    df_kanja.loc[:, ["確認日", "居住地", "年代", "性別", "備考欄"]].sort_index().reset_index()
)
df_pt.head(10)

df_pt["退院"] = None

df_pt["date"] = df_pt["確認日"].dt.strftime("%Y-%m-%d")
df_pt["リリース日"] = df_pt["確認日"].apply(
    lambda d: pd.Timestamp(d, tz="Asia/Tokyo").isoformat()
)

week = ["月", "火", "水", "木", "金", "土", "日"]

df_pt["曜日"] = df_pt["確認日"].dt.dayofweek.apply(lambda x: week[x])

df_pt["年代"] = df_pt["年代"].astype("str") + "代"

df_pt.rename(columns={"備考欄": "備考"}, inplace=True)

df_pt.drop("確認日", axis=1, inplace=True)

df_pt

,番号,居住地,年代,性別,備考,退院,date,リリース日,曜日
0,1,西宮市,40代,男性,特定できず,None,2020-03-01,2020-03-01T00:00:00+09:00,日
1,2,神戸市,40代,男性,特定できず,None,2020-03-03,2020-03-03T00:00:00+09:00,火
2,3,神戸市,40代,女性,2/15、16に大阪のライブハウスArcに参加,None,2020-03-03,2020-03-03T00:00:00+09:00,火
3,4,福崎町,50代,女性,2/15、16に大阪のライブハウスArcに参加,None,2020-03-05,2020-03-05T00:00:00+09:00,木
4,5,姫路市,40代,男性,2/19に大阪のSoap operaライブに参加,None,2020-03-06,2020-03-06T00:00:00+09:00,金
...,...,...,...,...,...,...,...,...,...
121,122,神戸市,10代,女性,海外からの帰国者,None,2020-03-27,2020-03-27T00:00:00+09:00,金
122,123,東京都,30代,女性,行動歴調査中,None,2020-03-27,2020-03-27T00:00:00+09:00,金
123,124,姫路市,40代,男性,行動歴調査中,None,2020-03-28,2020-03-28T00:00:00+09:00,土
124,125,伊丹健康福祉事務所管内,50代,男性,大阪府が3月26日に公表した患者(40代男性・会社員・大阪市居住)の濃厚接触者,None,2020-03-28,2020-03-28T00:00:00+09:00,土


In [0]:
patients = {
    "data": df_pt.to_dict(orient="records"),
    "last_update": last_update.strftime("%Y/%m/%d %H:%M"),
}

In [0]:
jsonschema.validate(patients, PATIENTS_SCHEMA)

dumps_json("patients.json", patients)

In [0]:
# 年代集計

age_list = [i for i in range(0, 100, 10)]

df_age = df_kanja["年代"].value_counts().sort_index().reindex(age_list, fill_value=0)

df_age.index = df_age.index.map(lambda s: str(s) + "代")
df_age.rename(index={"0代": "10代未満", "90代": "90代以上"}, inplace=True)

df_age = df_age.astype(int)

In [0]:
age = {
    "data": df_age.to_dict(),
    "last_update": last_update.strftime("%Y/%m/%d %H:%M"),
}

In [0]:
jsonschema.validate(age, AGE_SCHEMA)

dumps_json("age.json", age)

In [0]:
df_ages = pd.crosstab(df_kanja["確認日"], df_kanja["年代"]).reindex(
    age_list, axis=1, fill_value=0
)

df_ages.columns = df_ages.columns.map(lambda s: str(s) + "代")
df_ages.rename(columns={"0代": "10代未満", "90代": "90代以上"}, inplace=True)

df_ages = df_ages.astype(int)

df_agesum = df_ages.asfreq("D", fill_value=0)

df_agesum

年代,10代未満,10代,20代,30代,40代,50代,60代,70代,80代,90代以上
確認日,,,,,,,,,,
2020-03-01,0,0,0,0,1,0,0,0,0,0
2020-03-02,0,0,0,0,0,0,0,0,0,0
2020-03-03,0,0,0,0,2,0,0,0,0,0
2020-03-04,0,0,0,0,0,0,0,0,0,0
2020-03-05,0,0,0,0,0,1,0,0,0,0
2020-03-06,0,0,0,0,3,1,0,0,0,0
2020-03-07,0,0,0,0,1,0,0,0,1,0
2020-03-08,0,0,0,0,0,1,0,0,1,0
2020-03-09,0,0,0,1,1,1,0,0,1,0


In [0]:
labels = df_agesum.index.map(lambda d: d.strftime("%m/%d"))

In [0]:
age_summary = {
    "data": df_agesum.to_dict(orient="list"),
    "labels": labels.tolist(),
    "last_update": last_update.strftime("%Y/%m/%d %H:%M"),
}

In [0]:
jsonschema.validate(age_summary, AGE_SUMMARY_SCHEMA)

dumps_json("age_summary.json", age_summary)

In [0]:
# クラスタ概要

df_cluster_sum = df_kanja.loc[:, "認定こども園":].copy().notnull().sum()

In [0]:
clusters_summary = {
    "data": df_cluster_sum.to_dict(),
    "last_update": last_update.strftime("%Y/%m/%d %H:%M"),
}

In [0]:
jsonschema.validate(clusters_summary, CLUSTERS_SUMMARY_SCHEMA)

dumps_json("clusters_summary.json", clusters_summary)

In [0]:
# クラスタ

df_clusters = df_kanja.loc[:, "認定こども園":].copy().fillna(0)

df_clusters["確認日"] = df_kanja["確認日"]

df_clusters.replace("〇", 1, inplace=True)

pv_clusters = df_clusters.pivot_table(index="確認日", aggfunc="sum")

pv_clusters["日付"] = pv_clusters.index.strftime("%Y-%m-%d")

In [0]:
clusters = {
    "data": pv_clusters.to_dict(orient="recodes"),
    "last_update": last_update.strftime("%Y/%m/%d %H:%M"),
}

In [0]:
dumps_json("clusters.json", clusters)

In [0]:
# 重複者

(df_kanja.loc[:, "認定こども園":].copy().notnull().sum(axis=1) > 1).sum()

4

## yousei.xlsx

In [0]:
df_yousei = pd.read_excel("yousei.xlsx", index_col="発表年月日")

df_yousei.columns = df_yousei.columns.map(lambda s: s.replace("（累計）", "").strip())

df_yousei.index += pd.to_timedelta("1 days")

df_yousei.rename(columns={"中等症以下": "軽症・中等症", "陽性者数": "陽性患者数"}, inplace=True)

df_yousei.drop("発表時間", axis=1, inplace=True)

df_yousei

,検査実施人数,陽性患者数,入院中,軽症・中等症,重症,死亡,退院
発表年月日,,,,,,,
2020-03-11,444,25,24,20,4,0,1
2020-03-12,540,37,35,31,4,1,1
2020-03-13,650,46,44,40,4,1,1
2020-03-14,744,56,54,50,4,1,1
2020-03-15,824,67,64,57,7,1,2
2020-03-16,887,78,75,68,7,1,2
2020-03-17,951,82,78,72,6,2,2
2020-03-18,1024,86,81,75,6,2,3
2020-03-19,1127,91,84,78,6,3,4


In [0]:
d = df_yousei.iloc[-1].to_dict()

In [0]:
main_summary = {
    "attr": "検査実施人数",
    "value": d["検査実施人数"],
    "children": [
        {
            "attr": "陽性患者数",
            "value": d["陽性患者数"],
            "children": [
                {
                    "attr": "入院中",
                    "value": d["入院中"],
                    "children": [
                        {"attr": "軽症・中等症", "value": d["軽症・中等症"]},
                        {"attr": "重症", "value": d["重症"]},
                    ],
                },
                {"attr": "死亡", "value": d["死亡"]},
                {"attr": "退院", "value": d["退院"]},
            ],
        }
    ],
    "last_update": last_update.strftime("%Y/%m/%d %H:%M"),
}

In [0]:
jsonschema.validate(main_summary, MAIN_SUMMARY_SCHEMA)

dumps_json("main_summary.json", main_summary)

# チェック

In [0]:
json_check("patients_summary.json", patients_summary)
json_check("inspections_summary.json", inspections_summary)

patients_summary.json


In [0]:
json_check("patients_summary.json", patients_summary)
json_check("age.json", age)
json_check("age_summary.json", age_summary)
json_check("clusters.json", clusters)
json_check("clusters_summary.json", clusters_summary)
json_check("patients.json", patients)

patients_summary.json
clusters.json
patients.json


In [0]:
json_check("main_summary.json", main_summary)